In [1]:
import numpy as np
from qiskit import QuantumCircuit, execute
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import *
import qiskit

In [23]:
num_qubits = 32

alice_state = np.random.randint(0, 2, size=num_qubits)
eve_basis = np.random.randint(0, 2, size=num_qubits)
bob_basis = np.random.randint(0, 2, size=num_qubits)
print (alice_state)
print(f"Alice's State:\t {np.array2string(alice_state, separator='')}")
print(f"Eve's State:\t {np.array2string(eve_basis, separator='')}")
print(f"Bob's Basis:\t {np.array2string(bob_basis, separator='')}")
print (len(alice_state))

[1 1 1 0 0 1 1 0 0 1 0 1 1 1 1 1 0 0 0 1 1 0 1 0 1 1 0 1 0 1 0 1]
Alice's State:	 [11100110010111110001101011010101]
Eve's State:	 [01101111100101010011010101001011]
Bob's Basis:	 [00110101011111110011101000110011]
32


In [24]:
# function to define the circuit for an exchange of bit string
def make_b92_circ(enc_state, meas_basis):
    
    num_qubits = len(enc_state)

    b92 = QuantumCircuit(num_qubits)

    for index in range(len(enc_state)):
      if enc_state[index] == 1:
        b92.h(index)

    
    b92.barrier()  

    for index in range(len(meas_basis)):
      if meas_basis[index] == 1:
        b92.h(index)

    # Do not change below this line
    b92.measure_all()
    return b92

In [25]:
#Alice sends the bits to Bob but Eve eavesdrops in between
b92_circ = make_b92_circ(alice_state, eve_basis)

#final result got by Eve after measurement
eve_res = execute(
        b92_circ.reverse_bits(),
        backend=QasmSimulator(),
        shots=1, 
        seed_simulator=10
    ).result().get_counts().most_frequent()

print ("Eve's result:\t" + eve_res)
eve_state = np.array(list(eve_res), dtype=int)
print("Eve's result:\t" + np.array2string(eve_state, separator=''))
print(len(eve_state))

Eve's result:	00000000010010000000100010001010
Eve's result:	[00000000010010000000100010001010]
32


In [29]:
# Eve updates the state to send it to Bob
eve_state = ''.join([str(int(eve_res[i]) ^ int(eve_basis[i])) for i in range(len(eve_res))])

eve_state = np.array(list(eve_state), dtype=int)
print("Eve's result:\t" + np.array2string(eve_state, separator=''))

#Bob measures the circuit thinking that it was sent by Alice
b92_circ2 = make_b92_circ(eve_state, bob_basis)

bob_res = execute(
        b92_circ2.reverse_bits(),
        backend=QasmSimulator(),
        shots=1, 
        seed_simulator=10
    ).result().get_counts().most_frequent()

print (f"Bob's measured result:\t {bob_res}")

Eve's result:	[01101111110111010011110111000001]
Bob's measured result:	 00000000001000000000000010100010


In [30]:
def b92_sifting(enc_state, meas_basis, meas_result):
    
    num_qubits = len(enc_state)
    
    sender_key = ''
    receiver_key = ''
    
    for i in range(len(meas_result)):
      if meas_result[i] == '1':
        sender_key+=str(enc_state[i])
        receiver_key+=str((meas_basis[i]+1)%2)
        
    return (sender_key, receiver_key)

In [31]:
# key sifting takes place to get the final key to be used
alice_key, bob_key = b92_sifting(alice_state, bob_basis, bob_res)
print(f"Alice's State:\t {np.array2string(alice_state, separator='')}")
print(f"Eve's State:\t {np.array2string(eve_basis, separator='')}")
print(f"Bob's Basis:\t {np.array2string(bob_basis, separator='')}")
print(f"Eve's result:\t {np.array2string(eve_state, separator='')}")
print(f"Alice's final key: \t{alice_key}")
print(f"Bob's final key: \t{bob_key}")

Alice's State:	 [11100110010111110001101011010101]
Eve's State:	 [01101111100101010011010101001011]
Bob's Basis:	 [00110101011111110011101000110011]
Eve's result:	 [01101111110111010011110111000001]
Alice's final key: 	0100
Bob's final key: 	0100
